In [93]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [94]:
from langchain.text_splitter import TokenTextSplitter

In [95]:
# with open("/Users/sonhunseo/Desktop/공모전/4. 경기도_GenAI·공공데이터_창업경진대회/entire_content.txt") as f:
#     context = f.read()
# text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = text_splitter.split_text(context)

embeddings = OpenAIEmbeddings()

In [96]:
# texts[0]

'"집을 고르는 시점"에서의 전세사기의 유형\n\n1. 깡통주택\n\n깡통주택이란? :\n깡통주택이란, 임대인의 집이 경매로 넘어갔을 때, 내 보증금이 떼이게 되는 집을 통틀어 가리켜요. 위 그림처럼 매매가격의 대다수를 세입자의 보증금과 빚으로 채우고 있는 집이어서, 사실상 임대인의 몫은 거의 없는 집들이 흔히 깡통주택이에요.\n보통 깡통주택을 전세로 내놓는 임대인들은 ‘갭투기’를 통해 해당 주택을 사들였을 거예요. 이때의 갭(차이)은 ‘매매가-전세가’를 말해요. 그 ‘갭’마저도 은행에서 대출을 받고요. 이렇게 되면, 임대인이 스스로 자유롭게 융통할 수 있는 돈은 거의 없죠. 보증금도, 은행 대출금도 전부 언젠가 타인에게 돌려줘야 하는 돈이니까요. 그렇기 때문에 만약 집이 경매에 넘어가기라도 하면, 세입자는 보증금을 떼이게 되고요. 경매에 넘어가지 않더라도, 계약이 끝나고 이사를 나갈 때 제때 보증금을 돌려주지 못할 수도 있어요. 임대인이 �'

In [97]:
# docsearch = FAISS.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

In [102]:
# docsearch.save_local("entire_content_vectordb")

In [103]:
docsearch = FAISS.load_local("entire_content_vectordb", embeddings)

In [104]:
query = "전세계약을 하려는데 집주인이 아닌 집주인의 친척이라는 사람이 왔어 이거 사기 위험이 있는거 아니야?"
docs = docsearch.similarity_search(query)[0:1]

In [105]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

In [106]:
# template = """
# 질문자의 상황 : {question}
# =========
# {summaries}
# =========
# 질문자는 현재 본인이 전세사기의 위험이 있는지 알고싶어해.
# 너는 위 사례와 질문자의 상황의 비슷한점을 판단해서 어떠한점이 위험한지 알려줘야해
# 너는 반드시 답변시 질문자의 상황과 사례를 비교하면서 어떤점이 위험한지 알려줘야해
# """

# PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

# chain = load_qa_with_sources_chain(OpenAI(temperature=0, max_tokens=1500), chain_type="stuff", prompt=PROMPT)
# chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [109]:
template = """
Questioner status : {question}
=========
{summaries}
=========
The questioner wants to know if there is a risk of a charter fraud.
You should judge the similarities between the above case and the questioner's situation and let us know what is dangerous.
When you answer, you must compare the situation and case of the questioner and let them know what is dangerous.
you must answer IN KOREAN.
"""

PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(ChatOpenAI(temperature=0, max_tokens=1000), chain_type="stuff", prompt=PROMPT)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '질문자는 전세계약을 하려는데 집주인의 친척이 와서 사기 위험이 있는지 물어보았습니다. 이 경우, 집주인 대리인으로 나타난 사람이 진짜 집주인 대리인인지 확인하지 않고 계약을 체결하면 사기 위험이 있습니다. 이전에 있었던 사례와 비슷하게, 공인중개사가 세입자와 전세계약을 체결하면서 보증금을 빼돌리는 경우가 있습니다. 따라서, 집주인 대리인으로 나타난 사람이 정말로 집주인 대리인인지 사전에 확인하는 것이 중요합니다. 이를 위해서는 국가공간정보포털에서 해당 중개사가 등록되어 있는지 확인할 수 있습니다. 무자격자가 중개사무소를 차리는 경우도 있으므로, 등록된 중개업자인지 여부를 꼼꼼히 확인해야 합니다.'}